In [2]:
import numpy as np
import pandas as p

In [3]:
raw_sheets = p.read_csv('Remeasurements Master Sample Sheet - Sheet1.csv')

In [4]:
bonus_additions = {'080618':'Nextseq',
                  '082818':'Hiseq',
                  '090118':'Nextseq',
                  '092518_lane1':'Nextseq',
                  '092518_lane2':'Nextseq',
                   '120118':'NextseqHTP' 
                  }

In [5]:
# Primer_mapping

primer_renaming = {'P104': 'F201',
'P105': 'F202',
'P111': 'F203',
'P112': 'F204',
'P122': 'F205',
'P123': 'F206',
'P124': 'F207',
'P125': 'F208',
'P130': 'F209',
'P131': 'F210',
'P132': 'F211',
'P133': 'F212',
'P101': 'R301',
'P108': 'R302',
'P109': 'R303',
'P110': 'R304',
'P126': 'R305',
'P127': 'R306',
'P128': 'R307',
'P129': 'R308'}


In [5]:
for grouping in np.unique(raw_sheets['Groupings'].values):
    
    this_sample_sheet = raw_sheets[raw_sheets['Groupings']==grouping]
    print(this_sample_sheet)
    print(grouping)
    
    this_f = 'SampleSheets/' + 'Group_' + grouping + '_Samples.txt'
    
    with open(this_f,'w') as f:
        for entry in range(len(this_sample_sheet)):
            name = this_sample_sheet['ModifiedSample Name'].values[entry]
            
            name = name.replace(' ','_')
            
            
            lane_id = this_sample_sheet['Lane Identifier'].values[entry]
            
            if ('lane' in lane_id) and ('_' not in lane_id):
                lane_id = lane_id.replace('lane','_lane')
                
            if 'Diagonal' in lane_id:
                lane_id = lane_id.replace('Diagonal','diagonal')
                
            if 'pool' in lane_id:
                lane_id = lane_id.replace('pool','Experiment')
                
            if '6-7-8' in lane_id:
                lane_id = lane_id.replace('6-7-8','6-8')
            print(lane_id)
            
            
            
            if lane_id in bonus_additions:
                lane = lane_id + '_'+ bonus_additions[lane_id] + '_' + this_sample_sheet['Indexing'].values[entry]                
            else:
                lane = lane_id + '_' + this_sample_sheet['Indexing'].values[entry]
            if 'P' in str(this_sample_sheet['Forward Inline'].values[entry]):
                f_inline = str(primer_renaming[this_sample_sheet['Forward Inline'].values[entry]])
            else:
                f_inline = str(this_sample_sheet['Forward Inline'].values[entry])
            if 'P' in str(this_sample_sheet['Reverse Inline'].values[entry]):
                r_inline = str(primer_renaming[this_sample_sheet['Reverse Inline'].values[entry]])
            else:
                r_inline = str(this_sample_sheet['Reverse Inline'].values[entry])
            
            f.write(name + '\t' + lane + '\t' + f_inline + '\t' + r_inline + '\n' )

   Sample Name ModifiedSample Name  Lane Identifier   Indexing Forward Inline  \
0          3-0                 3-0  Experiment1and3  Unindexed           P112   
1        3-1-1               3-1-1  Experiment1and3  Unindexed           P111   
2        3-1-2               3-1-2  Experiment1and3  Unindexed           P112   
3        3-1-3               3-1-3  Experiment1and3  Unindexed           P105   
4        3-2-1               3-2-1  Experiment1and3  Unindexed           P123   
5        3-2-2               3-2-2  Experiment1and3  Unindexed           P112   
6        3-2-3               3-2-3  Experiment1and3  Unindexed           P122   
7        3-3-1               3-3-1  Experiment1and3  Unindexed           P123   
8        3-3-2               3-3-2  Experiment1and3  Unindexed           P124   
9        3-3-3               3-3-3  Experiment1and3  Unindexed           P104   
10       3-4-1               3-4-1  Experiment1and3  Unindexed           P105   
11       3-4-2              

In [27]:
# cores = 16

# for grouping in np.unique(raw_sheets['Groupings'].values):
#     sample_file = 'Group_' + grouping + '_Samples.txt'
#     group = 'Group_' + grouping
#     this_f = 'SampleSheets/' + 'Group_' + grouping + '_runner.sbatch'
#     with open(this_f,'w') as f:
#         f.write("""#!/usr/bin/env bash
# #SBATCH -J {group}
# #SBATCH -p hns,owners,dpetrov
# #SBATCH -n {cores}
# #SBATCH -N 1
# #SBATCH -t 7-00:00
# #SBATCH --array=1-1
# #SBATCH --mem-per-cpu=2G
# #SBATCH --requeue
# #SBATCH -o SlurmFiles/slurm-%A_%a_%x.out
# #SBATCH --mail-user=grantkinsler@gmail.com
# #SBATCH --mail-type=END

# module load python/3.6.1
# module load biology
# module load ncbi-blast+/2.7.1
# python3 /home/groups/dpetrov/SOFTWARE/BarcodeCounter2-master/barcodeCounter.py -fastqDir AllRawData/ -outputDir Output/ -templateSeq Template.txt -sample {sample_file} -multiBCFasta Primers.fasta -pairedEnd -useUMI -numThreads {cores} -demultiplexOnly -constRegionWriteFile  -barcodeList Consecutive_500pool_NoConstant.fa -bowtie2PATH /home/groups/dpetrov/SOFTWARE/bowtie2-2.2.6_new/""".format(group=group,sample_file=sample_file,cores=cores))
        
        

In [8]:
cores = 16

for grouping in np.unique(raw_sheets['Groupings'].values):
    sample_file = 'Group_' + grouping + '_Samples.txt'
    group = 'Group_' + grouping
    this_f = 'SampleSheets/' + 'Group_' + grouping + '_runner_newset.sbatch'
    with open(this_f,'w') as f:
        f.write("""#!/usr/bin/env bash
#SBATCH -J {group}
#SBATCH -p hns,dpetrov
#SBATCH -n {cores}
#SBATCH -N 1
#SBATCH -t 5-00:00
#SBATCH --array=1-1
#SBATCH --mem-per-cpu=2G
#SBATCH --requeue
#SBATCH -o SlurmFiles/slurm-%A_%a_%x.out
#SBATCH --mail-user=grantkinsler@gmail.com
#SBATCH --mail-type=END

module load python/3.6.1
module load biology
module load ncbi-blast+/2.7.1
python3 /home/groups/dpetrov/SOFTWARE/BarcodeCounter2-master_patched/barcodeCounter.py -fastqDir AllRawData/ -outputDir {group}_Output/ -templateSeq Template.txt -sample {sample_file} -multiBCFasta Primers_noP.fasta -pairedEnd -useUMI -numThreads {cores} -demultiplexOnly -barcodeList Consecutive_500pool_NoConstant.fa -bowtie2PATH /home/groups/dpetrov/SOFTWARE/bowtie2-2.2.6_new/""".format(group=group,sample_file=sample_file,cores=cores))
        
        

In [6]:
this_f = 'SampleSheets/' + 'All_Samples.txt'
    
with open(this_f,'w') as f:
    for grouping in np.unique(raw_sheets['Groupings'].values):
    
        this_sample_sheet = raw_sheets[raw_sheets['Groupings']==grouping]
        print(this_sample_sheet)
        print(grouping)
    

        for entry in range(len(this_sample_sheet)):
            name = this_sample_sheet['ModifiedSample Name'].values[entry]
            
            name = name.replace(' ','_')
            
            
            lane_id = this_sample_sheet['Lane Identifier'].values[entry]
            
            if ('lane' in lane_id) and ('_' not in lane_id):
                lane_id = lane_id.replace('lane','_lane')
                
            if 'Diagonal' in lane_id:
                lane_id = lane_id.replace('Diagonal','diagonal')
                
            if 'pool' in lane_id:
                lane_id = lane_id.replace('pool','Experiment')
                
            if '6-7-8' in lane_id:
                lane_id = lane_id.replace('6-7-8','6-8')
            print(lane_id)
            
            
            
            if lane_id in bonus_additions:
                lane = lane_id + '_'+ bonus_additions[lane_id] + '_' + this_sample_sheet['Indexing'].values[entry]                
            else:
                lane = lane_id + '_' + this_sample_sheet['Indexing'].values[entry]
            if 'P' in str(this_sample_sheet['Forward Inline'].values[entry]):
                f_inline = str(primer_renaming[this_sample_sheet['Forward Inline'].values[entry]])
            else:
                f_inline = str(this_sample_sheet['Forward Inline'].values[entry])
            if 'P' in str(this_sample_sheet['Reverse Inline'].values[entry]):
                r_inline = str(primer_renaming[this_sample_sheet['Reverse Inline'].values[entry]])
            else:
                r_inline = str(this_sample_sheet['Reverse Inline'].values[entry])
            
            f.write(name + '\t' + lane + '\t' + f_inline + '\t' + r_inline + '\n' )

   Sample Name ModifiedSample Name  Lane Identifier   Indexing Forward Inline  \
0          3-0                 3-0  Experiment1and3  Unindexed           P112   
1        3-1-1               3-1-1  Experiment1and3  Unindexed           P111   
2        3-1-2               3-1-2  Experiment1and3  Unindexed           P112   
3        3-1-3               3-1-3  Experiment1and3  Unindexed           P105   
4        3-2-1               3-2-1  Experiment1and3  Unindexed           P123   
5        3-2-2               3-2-2  Experiment1and3  Unindexed           P112   
6        3-2-3               3-2-3  Experiment1and3  Unindexed           P122   
7        3-3-1               3-3-1  Experiment1and3  Unindexed           P123   
8        3-3-2               3-3-2  Experiment1and3  Unindexed           P124   
9        3-3-3               3-3-3  Experiment1and3  Unindexed           P104   
10       3-4-1               3-4-1  Experiment1and3  Unindexed           P105   
11       3-4-2              

In [ ]:
cores = 16

for grouping in np.unique(raw_sheets['Groupings'].values):
    sample_file = 'Group_' + grouping + '_Samples.txt'
    group = 'Group_' + grouping
    this_f = 'SampleSheets/' + 'Group_' + grouping + '_runner_newset.sbatch'
    with open(this_f,'w') as f:
        f.write("""#!/usr/bin/env bash
#SBATCH -J {group}
#SBATCH -p hns,dpetrov
#SBATCH -n {cores}
#SBATCH -N 1
#SBATCH -t 7-00:00
#SBATCH --array=1-1
#SBATCH --mem-per-cpu=2G
#SBATCH --requeue
#SBATCH -o SlurmFiles/slurm-%A_%a_%x.out
#SBATCH --mail-user=grantkinsler@gmail.com
#SBATCH --mail-type=END

module load python/3.6.1
module load biology
module load ncbi-blast+/2.7.1
python3 /home/groups/dpetrov/SOFTWARE/BarcodeCounter2-master_patched/barcodeCounter.py -fastqDir AllRawData/ -outputDir {group}_Output/ -templateSeq Template.txt -sample {sample_file} -multiBCFasta Primers_noP.fasta -pairedEnd -useUMI -numThreads {cores} -demultiplexOnly -barcodeList Consecutive_500pool_NoConstant.fa -bowtie2PATH /home/groups/dpetrov/SOFTWARE/bowtie2-2.2.6_new/""".format(group=group,sample_file=sample_file,cores=cores))
        
        

In [12]:
cores = 16

for grouping in np.unique(raw_sheets['Groupings'].values):
    sample_file = 'Group_' + grouping + '_Samples.txt'
    group = 'Group_' + grouping
    this_f = 'SampleSheets/' + 'Group_' + grouping + '_runner_AncestorIncluded.sbatch'
    with open(this_f,'w') as f:
        f.write(f"""#!/usr/bin/env bash
#SBATCH -J {group}_AncInc
#SBATCH -p hns,dpetrov
#SBATCH -n {cores}
#SBATCH -N 1
#SBATCH -t 7-00:00
#SBATCH --array=1-1
#SBATCH --mem-per-cpu=2G
#SBATCH --requeue
#SBATCH -o SlurmFiles/slurm-%A_%a_%x.out
#SBATCH --mail-user=grantkinsler@gmail.com
#SBATCH --mail-type=END

module load python/3.6.1
module load biology
module load ncbi-blast+/2.7.1
python3 /home/groups/dpetrov/SOFTWARE/BarcodeCounter2-master_patched/barcodeCounter_grantedits.py -fastqDir AllRawData/ -outputDir {group}_AncestorIncluded_Output/ -templateSeq Template_double_longbc.txt -sample {sample_file} -multiBCFasta Primers_noP.fasta -pairedEnd -useUMI -numThreads 16 -demultiplexOnly -barcodeList Consecutive_500pool_NoConstant.fa -bowtie2PATH /home/groups/dpetrov/SOFTWARE/bowtie2-2.2.6_new/ -readLength 108""")
        

In [6]:
this_f = 'SampleSheets/' + 'No_F_Samples.txt'
    
with open(this_f,'w') as f:
    for grouping in np.unique(raw_sheets['Groupings'].values):
        if 'F' not in grouping:
    
            this_sample_sheet = raw_sheets[raw_sheets['Groupings']==grouping]
            print(this_sample_sheet)
            print(grouping)


            for entry in range(len(this_sample_sheet)):
                name = this_sample_sheet['ModifiedSample Name'].values[entry]

                name = name.replace(' ','_')


                lane_id = this_sample_sheet['Lane Identifier'].values[entry]

                if ('lane' in lane_id) and ('_' not in lane_id):
                    lane_id = lane_id.replace('lane','_lane')

                if 'Diagonal' in lane_id:
                    lane_id = lane_id.replace('Diagonal','diagonal')

                if 'pool' in lane_id:
                    lane_id = lane_id.replace('pool','Experiment')

                if '6-7-8' in lane_id:
                    lane_id = lane_id.replace('6-7-8','6-8')
                print(lane_id)



                if lane_id in bonus_additions:
                    lane = lane_id + '_'+ bonus_additions[lane_id] + '_' + this_sample_sheet['Indexing'].values[entry]                
                else:
                    lane = lane_id + '_' + this_sample_sheet['Indexing'].values[entry]
                if 'P' in str(this_sample_sheet['Forward Inline'].values[entry]):
                    f_inline = str(primer_renaming[this_sample_sheet['Forward Inline'].values[entry]])
                else:
                    f_inline = str(this_sample_sheet['Forward Inline'].values[entry])
                if 'P' in str(this_sample_sheet['Reverse Inline'].values[entry]):
                    r_inline = str(primer_renaming[this_sample_sheet['Reverse Inline'].values[entry]])
                else:
                    r_inline = str(this_sample_sheet['Reverse Inline'].values[entry])

                f.write(name + '\t' + lane + '\t' + f_inline + '\t' + r_inline + '\n' )

   Sample Name ModifiedSample Name  Lane Identifier   Indexing Forward Inline  \
0          3-0                 3-0  Experiment1and3  Unindexed           P112   
1        3-1-1               3-1-1  Experiment1and3  Unindexed           P111   
2        3-1-2               3-1-2  Experiment1and3  Unindexed           P112   
3        3-1-3               3-1-3  Experiment1and3  Unindexed           P105   
4        3-2-1               3-2-1  Experiment1and3  Unindexed           P123   
5        3-2-2               3-2-2  Experiment1and3  Unindexed           P112   
6        3-2-3               3-2-3  Experiment1and3  Unindexed           P122   
7        3-3-1               3-3-1  Experiment1and3  Unindexed           P123   
8        3-3-2               3-3-2  Experiment1and3  Unindexed           P124   
9        3-3-3               3-3-3  Experiment1and3  Unindexed           P104   
10       3-4-1               3-4-1  Experiment1and3  Unindexed           P105   
11       3-4-2              